In [2]:
import numpy as np
import pandas as pd
import sys
import os
import subprocess

import requests

import json


In [2]:
r = requests.get('https://financialmodelingprep.com/api/v3/historical-price-full/AAPL?serietype=line&apikey=b903b84d1ce1092d3e9fca34e25bf408')
a = r.json()
print(a)
print("SUCCESS")
print(pd.DataFrame(a))
print("Starting")


{'symbol': 'AAPL', 'historical': [{'date': '2021-04-16', 'close': 134.16}, {'date': '2021-04-15', 'close': 134.5}, {'date': '2021-04-14', 'close': 132.029999}, {'date': '2021-04-13', 'close': 134.429993}, {'date': '2021-04-12', 'close': 131.240005}, {'date': '2021-04-09', 'close': 133.0}, {'date': '2021-04-08', 'close': 130.360001}, {'date': '2021-04-07', 'close': 127.900002}, {'date': '2021-04-06', 'close': 126.209999}, {'date': '2021-04-05', 'close': 125.900002}, {'date': '2021-04-01', 'close': 123.0}, {'date': '2021-03-31', 'close': 122.150002}, {'date': '2021-03-30', 'close': 119.900002}, {'date': '2021-03-29', 'close': 121.389999}, {'date': '2021-03-26', 'close': 121.209999}, {'date': '2021-03-25', 'close': 120.589996}, {'date': '2021-03-24', 'close': 120.089996}, {'date': '2021-03-23', 'close': 122.540001}, {'date': '2021-03-22', 'close': 123.389999}, {'date': '2021-03-19', 'close': 119.989998}, {'date': '2021-03-18', 'close': 120.529999}, {'date': '2021-03-17', 'close': 124.7600

In [40]:
def getYLabels(priceDF):

    stock_data = priceDF

    names = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']

    print("Dimensions of file read: ")
    print("Rows")
    print(stock_data.shape[0])

    total_entries = stock_data.shape[0]


    #y-labels in following form:
    #(price_at_year_i, price_after_year_i, percentage change)

    #Initial time period = 1 year
    #On average 252 trading days so use current date + 252 entries to get approximate price one year later

    jsonYLabels = {}
    index = 1
    listDates = []
    listPriceOnDate = []
    listPriceAfterYear = []
    listPercentageChange = []
    mappingDateToIndex = {}
    for index2, dataPrice in stock_data.iterrows():
        if (index == 0):
            pass

        yLabelEntry = {}
        mapEntry = stock_data['historical'].iloc[index]
        current_date = mapEntry['date']
        current_price = float(mapEntry['close'])

        currentDateKey = current_date[0:7]

        mappingDateToIndex[str(currentDateKey)] = index

        #Instead of returning json, return mapping from date to index


        #Price a year later
        indexOneYearLater = index + 252
        #Check fence post to see if need to subtract 1 to avoid index out of range exception
        if (indexOneYearLater >= total_entries):
            break

        #If got here assume that indexOneYearLater not out of bounds
        mapEntryYearLater = stock_data['historical'].iloc[indexOneYearLater]
        price_after_year = float(mapEntryYearLater['close'])
        #price_after_year = float(stock_data['close'].iloc[indexOneYearLater])

        #print("Price after year")
        #print(price_after_year)

        #print("Current Price")
        #print(current_price)

        percentage_change = float((price_after_year - current_price)/current_price)
        #if (percentage_change < 0):
            #print("Negative percentage change")


        #Pass to json
        yLabelEntry['date'] = current_date
        yLabelEntry['price_on_date'] = current_price
        yLabelEntry['price_after_year'] = price_after_year
        yLabelEntry['percentage_change'] = percentage_change

        jsonYLabels[current_date] = yLabelEntry

        #Append to synchronous lists to create pandas dataframe
        listDates.append(current_date)
        listPriceOnDate.append(current_price)
        listPriceAfterYear.append(price_after_year)
        listPercentageChange.append(percentage_change)

        index = index + 1

        #check if new index (a year after) is below total_entries to see when to break

    #Create pandas table:
    d = {'date': listDates, 'price_on_date': listPriceOnDate, 'price_after_year': listPriceAfterYear, 'percentage_change': listPercentageChange}

    df = pd.DataFrame(data=d)

    #return jsonYLabels, df
    return mappingDateToIndex, df


In [4]:
res = []
stockTickers = ['BFOR', 'REZI', 'BSCM', 'VOSO', 'CSR', 'SOGO', 'SRL', 'CD', 'MDXG', 'AAPL', 'MSFT', 'FB', 'TSLA', 'BLK', 'CZR', 'ANF', 'ARAY', 'AVAV', 'AGFY']
for ticker in stockTickers:
    requestString = 'https://financialmodelingprep.com/api/v3/historical-price-full/' + str(ticker) + '?serietype=line&apikey=b903b84d1ce1092d3e9fca34e25bf408'
    r = requests.get(requestString)
    a = r.json()
    dataFrameCur = pd.DataFrame(a)
    print("Stock Name: "+ str(ticker))
    print(dataFrameCur)
    print("")
    print("")
    json_y_labels, pd_y_labels = getYLabels(dataFrameCur)
    res.append(pd_y_labels)

Stock Name: BFOR
     symbol                                  historical
0      BFOR      {'date': '2021-04-16', 'close': 58.74}
1      BFOR  {'date': '2021-04-15', 'close': 58.279999}
2      BFOR  {'date': '2021-04-14', 'close': 57.799999}
3      BFOR      {'date': '2021-04-13', 'close': 57.82}
4      BFOR  {'date': '2021-04-12', 'close': 58.130001}
...     ...                                         ...
1977   BFOR  {'date': '2013-06-10', 'close': 25.120001}
1978   BFOR  {'date': '2013-06-07', 'close': 25.059999}
1979   BFOR      {'date': '2013-06-06', 'close': 24.68}
1980   BFOR       {'date': '2013-06-05', 'close': 24.6}
1981   BFOR  {'date': '2013-06-04', 'close': 24.879999}

[1982 rows x 2 columns]


Dimensions of file read: 
Rows
1982
Stock Name: REZI
    symbol                                  historical
0     REZI      {'date': '2021-04-16', 'close': 28.57}
1     REZI  {'date': '2021-04-15', 'close': 28.969999}
2     REZI      {'date': '2021-04-14', 'close': 28.59}
3     REZI 

In [5]:
combined = pd.concat(res)


In [6]:
print("Combined y labels: ")
print(combined)

Combined y labels: 
            date  price_on_date  price_after_year  percentage_change
0     2021-04-15      58.279999         32.880001          -0.435827
1     2021-04-14      57.799999         34.040001          -0.411073
2     2021-04-13      57.820000         33.330002          -0.423556
3     2021-04-12      58.130001         34.070000          -0.413900
4     2021-04-09      58.000000         33.250000          -0.426724
...          ...            ...               ...                ...
3326  2008-01-29      23.260000         22.879999          -0.016337
3327  2008-01-28      23.790001         23.160000          -0.026482
3328  2008-01-25      23.299999         23.600000           0.012876
3329  2008-01-24      23.049999         23.900000           0.036876
3330  2008-01-23      23.590000         23.930000           0.014413

[66408 rows x 4 columns]


In [3]:
import requests 
import pandas as pd 

class API:
    def __init__(self, key, period='annual', limit=5):
        self.base_url = 'https://financialmodelingprep.com/'
        self.params = {
            'apikey': key,
            'period': period,
            'limit': limit,
        }
        self.alwaysCols = ["symbol", "date"]

    def request(self, url):
        r = requests.get(url, params=self.params)

        if r.status_code != 200:
            raise Exception('API did not return a valid response')

        return r

    def getDataFrame(self, endpoint): 
        r = self.request(endpoint)

        try:
            df = pd.json_normalize(r.json())
        except ValueError:
            raise Exception("Bad URL or API Key!")

        # df = df.reindex(index=df.index[::-1])
        df.sort_values(by=['date'], inplace=True, ascending=True)
        df.reset_index(drop=True, inplace=True)
        return df

    def getRatios(self, ticker):
        endpoint = self.base_url + 'api/v3/ratios/' + ticker
        wantedCols = [
            "currentRatio", "quickRatio", "grossProfitMargin", "operatingProfitMargin", "returnOnAssets",
            "returnOnEquity", "returnOnCapitalEmployed", "debtEquityRatio", "priceToBookRatio", "priceToSalesRatio",
            "priceEarningsRatio", "priceEarningsToGrowthRatio", "priceToOperatingCashFlowsRatio" 
        ]
        df = self.getDataFrame(endpoint)
        df = df[self.alwaysCols + wantedCols]
        return df

    def getGrowths(self, ticker):
        endpoint = self.base_url + 'api/v3/income-statement-growth/' + ticker
        wantedCols = [
            "growthRevenue", "growthCostOfRevenue", "growthGrossProfit", "growthGrossProfitRatio",
            "growthOperatingExpenses", "growthEBITDA", "growthOperatingIncome", "growthNetIncome",
            "growthEPS"
        ]
        df = self.getDataFrame(endpoint)
        df = df[self.alwaysCols + wantedCols]
        return df

    def getValues(self, ticker):
        endpoint = self.base_url + 'api/v3/enterprise-values/' + ticker
        wantedCols = [
            "marketCapitalization", "enterpriseValue"
        ]
        df = self.getDataFrame(endpoint)
        df = df[self.alwaysCols + wantedCols]
        return df
    
    def getFinStat(self, ticker, limit):
        self.params['limit'] = limit
        self.params['period'] = 'quarter'
        endpoint = self.base_url + 'api/v3/income-statement/' + ticker
        return self.getDataFrame(endpoint)

    def getSamples(self, ticker):
        ratios = self.getRatios(ticker)
        growths = self.getGrowths(ticker)
        values = self.getValues(ticker)

        merge = pd.merge(ratios, growths, how='inner', left_on=["date", "symbol"], right_on=["date", "symbol"])
        merge = pd.merge(merge, values, how='inner', left_on=["date", "symbol"], right_on=["date", "symbol"])

        return merge

Get respective X Data:


In [27]:
def getXData(stockTicker):
  api_key = 'b903b84d1ce1092d3e9fca34e25bf408'
  api = API(api_key, period='quarter', limit='52')
  XData = api.getSamples(stockTicker)
  print(gmeXData)
  return XData

getXData('GME')


   symbol        date  ...  marketCapitalization  enterpriseValue
0     GME  2008-05-03  ...          6.085241e+09     6.004241e+09
1     GME  2008-08-02  ...          4.508022e+09     4.513322e+09
2     GME  2008-11-01  ...          3.098206e+09     3.165606e+09
3     GME  2009-01-31  ...          2.802979e+09     2.770550e+09
4     GME  2009-05-02  ...          3.318773e+09     3.584073e+09
5     GME  2009-08-01  ...          2.474594e+09     2.772494e+09
6     GME  2009-10-31  ...          2.693619e+09     2.848713e+09
7     GME  2010-01-30  ...          2.230940e+09     1.772840e+09
8     GME  2010-05-01  ...          2.561576e+09     2.577276e+09
9     GME  2010-07-31  ...          2.041830e+09     2.200280e+09
10    GME  2010-10-30  ...          2.028205e+09     2.096005e+09
11    GME  2011-01-29  ...          2.148534e+09     1.686734e+09
12    GME  2011-04-30  ...          2.467326e+09     2.320726e+09
13    GME  2011-07-30  ...          2.257624e+09     2.292124e+09
14    GME 

,symbol,date,currentRatio,quickRatio,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,debtEquityRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceEarningsToGrowthRatio,priceToOperatingCashFlowsRatio,growthRevenue,growthCostOfRevenue,growthGrossProfit,growthGrossProfitRatio,growthOperatingExpenses,growthEBITDA,growthOperatingIncome,growthNetIncome,growthEPS,marketCapitalization,enterpriseValue
0,GME,2008-05-03,1.529371,0.601041,0.261028,0.059330,0.016482,0.031200,0.041143,0.892986,3.057296,3.355338,24.497750,-0.363687,-30.320085,-0.367113,-0.385652,-0.307996,0.093412,-0.064211,-0.604709,-0.633015,-0.672813,-0.673594,6.085241e+09,6.004241e+09
1,GME,2008-08-02,1.595597,0.555823,0.268289,0.055476,0.015163,0.027716,0.037194,0.827890,2.184331,2.498349,19.702892,-2.380841,-474.528605,-0.005073,-0.014849,0.022602,0.027817,0.050027,-0.033195,-0.069703,-0.078905,-0.082756,4.508022e+09,4.513322e+09
2,GME,2008-11-01,1.465754,0.360022,0.279177,0.050304,0.011216,0.022624,0.031652,1.017101,1.500923,1.827095,16.585683,-0.906022,469.425091,-0.060242,-0.074226,-0.022103,0.040583,0.010414,-0.141657,-0.147852,-0.183566,-0.183061,3.098206e+09,3.165606e+09
3,GME,2009-01-31,1.163389,0.412182,0.240306,0.105323,0.051484,0.101025,0.124683,0.962268,1.218856,0.802660,3.016226,0.007588,3.742290,1.059394,1.170447,0.772659,-0.139233,0.214576,2.942876,3.311841,3.974839,3.974771,2.802979e+09,2.770550e+09
4,GME,2009-05-02,1.241646,0.225709,0.273677,0.063409,0.016560,0.029121,0.041572,0.758511,1.372812,1.675471,11.785415,-0.169094,-12.351220,-0.432779,-0.457733,-0.354010,0.138867,-0.116206,-0.645960,-0.658510,-0.696976,-0.696974,3.318773e+09,3.584073e+09
5,GME,2009-08-01,1.318153,0.217213,0.284958,0.040840,0.009038,0.015016,0.022282,0.661571,0.960187,1.423408,15.985750,-0.355262,313.239766,-0.122324,-0.135896,-0.086146,0.041220,0.018723,-0.370538,-0.434713,-0.450284,-0.449970,2.474594e+09,2.772494e+09
6,GME,2009-10-31,1.193521,0.187436,0.285102,0.049200,0.010370,0.019787,0.028230,0.907996,1.020533,1.468131,12.894298,0.332031,14.407769,0.055351,0.055139,0.055882,0.000505,0.019828,0.507671,0.271394,0.349483,0.388347,2.693619e+09,2.848713e+09
7,GME,2010-01-30,1.284834,0.585493,0.248014,0.098581,0.043570,0.079282,0.105285,0.819697,0.819235,0.633070,2.583303,0.008538,3.104565,0.920722,1.020367,0.670860,-0.130086,0.216917,1.810123,2.848497,3.134035,3.025594,2.230940e+09,1.772840e+09
8,GME,2010-05-01,1.350786,0.364011,0.274067,0.059730,0.016581,0.028057,0.038277,0.692087,0.955705,1.229931,8.515879,-0.135822,-9.932440,-0.408995,-0.429472,-0.346911,0.105046,-0.152459,-0.601715,-0.641911,-0.651691,-0.626988,2.561576e+09,2.577276e+09
9,GME,2010-07-31,1.373727,0.289028,0.287270,0.038701,0.009247,0.015264,0.021700,0.651172,0.772557,1.134922,12.652936,-0.288816,-61.323571,-0.136173,-0.151884,-0.094559,0.048175,0.001790,-0.242147,-0.440297,-0.463524,-0.438097,2.041830e+09,2.200280e+09


In [13]:
print(len(gmeXData.columns))

26


In [58]:
def getTrainingData(stockTicker):

  #Y Data for given stock
  requestString = 'https://financialmodelingprep.com/api/v3/historical-price-full/' + str(stockTicker) + '?serietype=line&apikey=b903b84d1ce1092d3e9fca34e25bf408'
  r = requests.get(requestString)
  a = r.json()
  dataFrameCur = pd.DataFrame(a)
  #print("Stock Name: "+ str(stockTicker))
  #print(dataFrameCur)

  mappingDateToIndex, pd_y_labels = getYLabels(dataFrameCur)

  #print('Data in JSON format')
  #print(json_y_labels)

  #First return value is now mapping from date to index of y labels
  #print('Mapping from Date to Index')
  #print(mappingDateToIndex)

  #print('Data in Data Frame format')
  #print(pd_y_labels)

  

  

  #Get X Data for given stock
  XDataCur = getXData(str(stockTicker))

  filteredYData = []
  filteredXData = []
  for index, row in XDataCur.iterrows():

    #Date must be in x and y data

    if (index > 1):
      pass
    print("Row")
    print(row)
    dateXValue = row['date']
    dateMonth = dateXValue[0:7]
    #print(dateXValue)
    #print('Testing')
    #print('2021-04-15' in mappingDateToIndex)

    if (dateMonth in mappingDateToIndex):
      print("Inside loop")
      filteredXData.append(row)
    
      yLabelIndex = mappingDateToIndex[str(dateMonth)]
      #Get y label row given appropiate index 
      yLabelRow = pd_y_labels.iloc[yLabelIndex]
      print("Date")
      print(dateXValue)
      #print("Y Row for date")
      #print(yLabelRow)
      print("Percentage change for y Row: ")
      percentageChange = yLabelRow['percentage_change']
      print(percentageChange)

      filteredYData.append(percentageChange)

  
  #FilteredYData will have percentage change as only y value 
  return filteredXData, filteredYData






  

filteredXData, filteredYData = getTrainingData('GME')

print("Filtered Y Data")
#print(filteredYData)
print(len(filteredYData))

print("Filtered X Data")
#print(filteredXData)
print(len(filteredXData))



Dimensions of file read: 
Rows
4827
   symbol        date  ...  marketCapitalization  enterpriseValue
0     GME  2008-05-03  ...          6.085241e+09     6.004241e+09
1     GME  2008-08-02  ...          4.508022e+09     4.513322e+09
2     GME  2008-11-01  ...          3.098206e+09     3.165606e+09
3     GME  2009-01-31  ...          2.802979e+09     2.770550e+09
4     GME  2009-05-02  ...          3.318773e+09     3.584073e+09
5     GME  2009-08-01  ...          2.474594e+09     2.772494e+09
6     GME  2009-10-31  ...          2.693619e+09     2.848713e+09
7     GME  2010-01-30  ...          2.230940e+09     1.772840e+09
8     GME  2010-05-01  ...          2.561576e+09     2.577276e+09
9     GME  2010-07-31  ...          2.041830e+09     2.200280e+09
10    GME  2010-10-30  ...          2.028205e+09     2.096005e+09
11    GME  2011-01-29  ...          2.148534e+09     1.686734e+09
12    GME  2011-04-30  ...          2.467326e+09     2.320726e+09
13    GME  2011-07-30  ...          2.25

In [59]:
res = []
stockTickers = ['BFOR', 'REZI', 'BSCM', 'VOSO', 'CSR', 'SOGO', 'SRL', 'CD', 'MDXG', 'AAPL', 'MSFT', 'FB', 'TSLA', 'BLK', 'CZR', 'ANF', 'ARAY', 'AVAV', 'AGFY']
XData = []
YData = []

#Length of two should be 52 * 20 
for ticker in stockTickers:
  print("Stock Ticker: ")
  print(ticker)
  curX, curY = getTrainingData(str(ticker))
  XData.append(curX)
  YData.append(curY)

#X Combined
XFinal = pd.concat(XData)

print("X Data final length")
print(len(XFinal))

YFinal = pd.concat(YData)

print("Y Data final length")
print(len(YFinal))

#use pd.concat




Stock Ticker: 
BFOR
Dimensions of file read: 
Rows
1982


KeyError: ignored

After making sure X and Y values have same dimension, run ML model